In [ ]:
import requests

In [ ]:
class BizError(Exception):
    pass

In [ ]:
###############################################################
#               添加 OMS 用户脚本使用方法
#     1. 填写用户信息 account (一般为邮箱前缀) 和 accountName
#     2. 填写与哪个用户权限相同 refAccount (一般为邮箱前缀)
#     3. 填写 ssoid 这个需要你先登录到 OMS dashboard 
#        然后从 cookie 里面扣除来 key = OMSSESSIONID 的字段
#     4. 选择菜单 Cell ==> Run All 执行脚本
#    
#     example :
#     account = 'test.beibei1' 
#     accountName = '测试用户' 
#     refAccount = 'wayne.wang111111' 
#     ssoid = 'd6cc0089-60b3-4a09-b0df-3dd01f316f08';
#
#     无视这条 host = 'http://oms-dev.int.husor.com:7777' 
###############################################################
account = 'yufang.chen' 
accountName = '陈玉芳' 
refAccount = 'wenqi.zhang' 

ssoid = 'd7ea999a-02fd-4b9e-89da-458b41a74356';

host = 'http://oms.beibei.com.cn'

#################### 以下代码都是业务逻辑，一般情况下不需要修改 ########################
# 清楚数据中可能存在的空格
ssoid = ssoid.strip()
account = account.strip()
accountName = accountName.strip()
refAccount = refAccount.strip()

cookies = {}
cookies['OMSSESSIONID'] = ssoid
# raise ValueError('sso id not valid')
# 获取参照权限 position
# refUser = requests.get('', cookies=cookies).json()
param = {}
param['username'] = account;
res = requests.get(host + '/api/users/username', params=param, cookies=cookies)

try:
    user = res.json()
    if (user is not None and user['success'] and user['data']):
        raise BizError(account + ' has been grant position = ' + str(user['data']['positionId']))
except ValueError as ve:
    print('sso id not valid')
    raise ve
except BizError as be:
    print(be)
    raise be
except Exception as e:
    raise e


param['username'] = refAccount;
res = requests.get(host + '/api/users/username', params=param, cookies=cookies)
try:
    user = res.json()
    if (user is None or not user['success'] or user['data'] is None):
        raise BizError(refAccount + ' do not exist!')
    position = user['data']['positionId']
    if (position is None):
        raise BizError(refAccount + ' do not has position')
except BizError as be:
    print(be)
    raise be
except Exception as e:
    raise e

url = host + '/api/users/addUsers?userNames=' + account + ':' + accountName + '&position=' + str(position)
res = requests.post(url, cookies=cookies)
print(res.text)
if (res.json() and res.json()['success']):
    print('OMS user {} has been add'.format(account))
else:
    print('OMS user add failed')